<a href="https://colab.research.google.com/github/snowboarderhk/practice/blob/master/GBDT_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#======SIGNATE BEGINNER COMPETITION 21/03==========
# 参考https://www.codexa.net/kaggle-titanic-beginner/

#---ライブラリのインポート---
import pandas as pd
import numpy as np
from sklearn import tree

#---データセット読み込み---
train = pd.read_csv("/content/train.csv")
test  = pd.read_csv("/content/test.csv")

#---データセットのサイズ確認---
test_shape = test.shape
train_shape = train.shape
#print(test_shape,train_shape)

#---データセットの内容確認---
print(train.head())
#print("===========================================================================================")
#print(test.head())
#カテゴリカルなデータは、workclass,education,marital-status,occupation,relationship,race,sex,native-country

#----欠損データの確認----
def kesson_table(df):
    null_val = df.isnull().sum() #isnull()は欠損データか否かを返す
    percent = 100* df.isnull().sum()/len(df)
    kesson_table = pd.concat([null_val,percent],axis=1)#concat は連結、引数のaxisは連結方向
    #print(kesson_table)
    kesson_table_ren_columns = kesson_table.rename( columns = {0:"欠損数",1:"%"})
    return kesson_table_ren_columns

#print(kesson_table(test))  
#欠損データなし21.03.01確認済み

   index  Pregnancies  Glucose  ...  DiabetesPedigreeFunction  Age  Outcome
0    200            9      125  ...                  0.444902   45        1
1   3832            4      109  ...                  0.238243   22        0
2   4927            4       88  ...                  0.279471   26        0
3   4088            9      125  ...                  0.203922   45        0
4   3644            5      107  ...                  0.284959   45        1

[5 rows x 10 columns]


In [ ]:
#---データセットの内容確認---
check_list = ["Pregnancies","Glucose","BloodPressure","SkinThickness","Insulin","BMI","DiabetesPedigreeFunction","Age"]

train["BloodPressure"][train["BloodPressure"]== 0] = train["BloodPressure"].median()

content = []
for i in check_list:
    content.append(train[i].value_counts()) #参考https://note.nkmk.me/python-pandas-value-counts/
    #content.append(train[i].unique()) #各カラムに含まれる内容を一覧にする。
print(content) 
"""
Pregnancies : ninshin times 
Glucose : kettou budoto nodo
DiabetesPedigreeFunction : tounyo byou kettou?
blood presssure 0 ? ->推定するべきかも

"""


[1     513
2     479
0     430
3     322
4     266
5     228
6     208
7     172
8     140
9      92
10     86
13     28
11     26
12     10
Name: Pregnancies, dtype: int64, 100    225
123    182
125    125
111    119
95      98
      ... 
149      1
179      1
76       1
187      1
73       1
Name: Glucose, Length: 105, dtype: int64, 70     333
74     292
68     278
64     251
80     224
60     223
62     196
72     174
76     146
78     133
88     124
66     105
82      96
84      74
58      66
86      60
90      52
65      49
54      34
56      23
85      12
96      10
52      10
50      10
48       6
110      5
75       4
46       4
92       3
94       1
100      1
106      1
Name: BloodPressure, dtype: int64, 0     1766
19     106
30      87
18      86
33      79
29      77
15      69
32      62
37      61
27      59
22      55
17      52
39      51
36      51
20      48
25      47
31      37
28      26
23      25
21      22
41      17
44      16
26      15
45      14
40      13
4

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


'\nPregnancies : ninshin times \nGlucose : kettou budoto nodo\nDiabetesPedigreeFunction : tounyo byou kettou?\nblood presssure 0 ? ->推定するべきかも\n\n'

In [ ]:
#---trainデータセットを目的変数と説明変数に分ける---
target = train["Outcome"].values

features_one = train[["Pregnancies","Glucose","BloodPressure","SkinThickness","Insulin","BMI","DiabetesPedigreeFunction","Age"]].values
#features_one = train[["Pregnancies","Glucose","SkinThickness","Insulin","BMI","DiabetesPedigreeFunction","Age"]].values

In [ ]:
#---testデータセットから説明変数の値を取得---
test_features = test[["Pregnancies","Glucose","BloodPressure","SkinThickness","Insulin","BMI","DiabetesPedigreeFunction","Age"]].values
#test_features = test[["Pregnancies","Glucose","SkinThickness","Insulin","BMI","DiabetesPedigreeFunction","Age"]].values

In [ ]:
#正規化
from sklearn import preprocessing
ss = preprocessing.StandardScaler()

features_one_std = ss.fit_transform(features_one)
test_features_std = ss.fit_transform(test_features)

In [ ]:
import xgboost as xgb
from sklearn.model_selection import cross_val_score

In [ ]:
depth_list = [3,4,5,6]
sub_list = [0.7,0.8,0.9,1.0]
col_list = [0.7,0.8,0.9,1.0]

for i in depth_list:
  for j in sub_list:
    for k in col_list:
      clf = xgb.XGBClassifier(objective="binary:logistic",max_depth = i,subsample = j,colsample_bytree = k)
      scores = cross_val_score(clf,features_one,target,cv=5)
      print("depth=",i,"sub=",j,"col=",k,"score=",scores.mean())

depth= 3 sub= 0.7 col= 0.7 score= 0.804
depth= 3 sub= 0.7 col= 0.8 score= 0.8046666666666666
depth= 3 sub= 0.7 col= 0.9 score= 0.8063333333333332
depth= 3 sub= 0.7 col= 1.0 score= 0.8083333333333332
depth= 3 sub= 0.8 col= 0.7 score= 0.8030000000000002
depth= 3 sub= 0.8 col= 0.8 score= 0.8029999999999999
depth= 3 sub= 0.8 col= 0.9 score= 0.8043333333333333
depth= 3 sub= 0.8 col= 1.0 score= 0.8016666666666667
depth= 3 sub= 0.9 col= 0.7 score= 0.807
depth= 3 sub= 0.9 col= 0.8 score= 0.8023333333333333
depth= 3 sub= 0.9 col= 0.9 score= 0.8049999999999999
depth= 3 sub= 0.9 col= 1.0 score= 0.8016666666666667
depth= 3 sub= 1.0 col= 0.7 score= 0.8016666666666667
depth= 3 sub= 1.0 col= 0.8 score= 0.8043333333333333


KeyboardInterrupt: ignored

In [ ]:
#モデルを使った予測
clf = xgb.XGBClassifier(objective="binary:logistic",max_depth = 5,subsample=0.9,colsample_bytree = 0.8)
clf.fit(features_one,target)
print(clf.score(features_one,target))
pred = clf.predict(test_features)

0.9063333333333333


In [ ]:
#モデルを使った予測
pred = clf.predict(test_features)

In [ ]:
#特徴量の影響度確認
importance = clf.feature_importances_
print('Feature Importances:')
for i, feat in enumerate(check_list):
  print(feat,"=",importance[i])

"""
どの特徴量も影響ありそう。 -> 21.03.05. blood pressure の0はおかしいからまずは平均値を入れてみる？
Feature Importances:
Pregnancies = 0.15181226
Glucose = 0.10986183
BloodPressure = 0.10303249
SkinThickness = 0.105441645
Insulin = 0.08439745
BMI = 0.15092531
DiabetesPedigreeFunction = 0.099175155
Age = 0.19535388
"""

Feature Importances:
Pregnancies = 0.14527072
Glucose = 0.1137626
BloodPressure = 0.09961415
SkinThickness = 0.10341836
Insulin = 0.08522641
BMI = 0.15781423
DiabetesPedigreeFunction = 0.09702948
Age = 0.19786403


'\nどの特徴量も影響ありそう。 -> 21.03.05. blood pressure の0はおかしいからまずは平均値を入れてみる？\nFeature Importances:\nPregnancies = 0.15181226\nGlucose = 0.10986183\nBloodPressure = 0.10303249\nSkinThickness = 0.105441645\nInsulin = 0.08439745\nBMI = 0.15092531\nDiabetesPedigreeFunction = 0.099175155\nAge = 0.19535388\n'

In [ ]:
# indexを取得
index = np.array(test["index"]).astype(int)

# my_predictionとPassengerIdをデータフレームへ落とし込む
my_solution = pd.DataFrame(pred,index,columns = ["Outcome"])
print(my_solution)
# my_tree_one.csvとして出力
my_solution.to_csv("/content/first_GBDT_5.csv",index_label = ["index"])

      Outcome
398         0
3833        0
4836        0
4572        1
636         0
...       ...
3138        0
191         0
3294        1
3073        0
361         0

[2000 rows x 1 columns]
